In [1]:
import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

In [2]:
from collections import Iterable, deque
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from bnelearn.strategy import NeuralNetStrategy, TruthfulStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import Environment

In [4]:
torch.cuda.is_available()

False

In [5]:
hidden_layer = 10
batch_size = 5
u_lo = 0
u_hi = 10
n_players = 2
n_items = 1

epochs = 1

In [6]:
model = NeuralNetStrategy(2, hidden_layer).cuda()

RuntimeError: cuda runtime error (30) : unknown error at ..\aten\src\THC\THCGeneral.cpp:74

In [ ]:
mechanism = FirstPriceSealedBidAuction(cuda=True)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [ ]:
p1 =Bidder.uniform(u_lo,u_hi, model, batch_size = batch_size, n_players=2)

In [ ]:
p2 = Bidder.uniform(u_lo,u_hi, model, batch_size = batch_size, n_players=2)

In [ ]:
p1.draw_valuations_(), p2.draw_valuations_();

In [ ]:
b1 = p1.get_action()
b2 = p2.get_action()

In [ ]:
for epoch in range(epochs):
    # draw new valuations and bids
    p1.draw_valuations_(), p2.draw_valuations_()    
    b1 = p1.get_action()
    b2 = p2.get_action()    
    bids = torch.cat((b1,b2), 1).view(batch_size, n_players, 1)
    
    # run the auction and calculate utilities
    alloc, pay = mechanism.run(bids)
    
    u1 = p1.get_utility(alloc[:,0,:], pay[:,0]).mean()
    u2 = p2.get_utility(alloc[:,1,:], pay[:,1]).mean()

    loss = -u2
    
    if epoch % 50 == 0:
        print('epoch ', epoch, 'avg bid', (b1/p1.valuations).mean().item(), (b2/p2.valuations).mean().item(), 'utility', u1.item(), u2.item() )
    
    # propagate loss and update parameters   
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# Same with ES

In [ ]:
u_lo =0
u_hi =10

In [ ]:
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [ ]:
#model = NeuralNetStrategy(2, hidden_layer).cuda()
#model = TruthfulStrategy(2).cuda()

In [ ]:
#mechanism = FirstPriceSealedBidAuction(cuda=True)
#mechanism = VickreyAuction(cuda=True)

In [ ]:
batch_size = 2**14

In [ ]:
env = Environment(mechanism, [], max_env_size=1, batch_size=batch_size,
                  n_players=2, strategy_to_bidder_closure=strat_to_bidder)


In [ ]:
#static_env = Environment(mechanism, [Bidder.uniform(u_lo,u_hi, TruthfulStrategy(2), batch_size=batch_size, n_players=2)],
#                         max_env_size=1, batch_size=batch_size, n_players=2, strategy_to_bidder_closure=strat_to_bidder)

In [ ]:
es = ES(model=model, environment=env, lr=5e-3, sigma=.1, n_perturbations= 200)


#es = ES(model=model, environment=static_env, lr=1e-2, sigma=.1, n_perturbations= 64, env_type='fixed')


In [ ]:
from tensorboardX import SummaryWriter

In [ ]:
writer = SummaryWriter()

In [ ]:
epoch=5001
start_epoch=0
plot_points = min(200, batch_size)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for e in range(epoch):
    if e%100==0:        
        bidder = strat_to_bidder(model, batch_size)
        bidder.draw_valuations_()
        v = bidder.valuations.detach().cpu().numpy()[:plot_points]
        b = bidder.get_action().detach().cpu().numpy()[:plot_points]
        optimal = v/2
        share = (bidder.get_action() / bidder.valuations).mean()
        writer.add_graph(model, torch.cat((bidder.valuations,
                                           torch.zeros_like(bidder.valuations)+2),
                                          dim=1))
        print("Epoch {}: \tavg bid: {:2f}, \tutility: {:2f} ".format(
            start_epoch + e, share.item(),-es.step().item())
             )
        plt.plot(v,b, 'o', v, optimal, 'r-')
        plt.show()
    else: es.step()    

In [ ]:
model.forward(torch.tensor([[5.,2]], device='cuda'))

# two truthful agents meet

In [ ]:
del es

In [ ]:
batch_size=500000
p1 = Bidder.uniform(0,10, strategy=TruthfulStrategy(1), batch_size=batch_size)
p2 = Bidder.uniform(0,10, strategy=TruthfulStrategy(1), batch_size=batch_size)

p1.draw_valuations_()
p2.draw_valuations_()
b1 = p1.get_action()
b2 = p2.get_action()

bids = torch.cat((b1,b2), 1).view(batch_size, n_players, 1)
    
    # run the auction and calculate utilities
alloc, pay = mechanism.run(bids)
    
u1 = p1.get_utility(alloc[:,0,:], pay[:,0]).mean()
u2 = p2.get_utility(alloc[:,1,:], pay[:,1]).mean()
print("utilities: {:.4f}, {:.4f}".format(u1.item(), u2.item()))

In [ ]:
2**19

In [ ]:
es.environment.agents[0].valuations[:10]